In [1]:
import pandas as pd
import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import ast
import csv
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
path = "./inf_narrative_msc/k24086575/dot_product/neuron_contribution_between_class.csv"
df = pd.read_csv(path, encoding="utf-8-sig")
df.sort_values("Contribution", ascending=False).reset_index(drop=True)
df.head()

,Unnamed: 0,type,layer,class1,class2,Neuron_ID,Contribution
0,0,mlp,7,math,programming,6117,8.144039e+11
1,1,mlp,24,math,programming,9329,7.782393e+11
2,2,mlp,23,math,programming,10184,7.565522e+11
3,3,res,1,math,programming,12054,6.470184e+11
4,4,mlp,24,math,programming,282,5.486137e+11


In [3]:
df.shape

(2354851, 7)

In [31]:
import pandas as pd

def get_X_contribution(p):
    """
    Returns (idx, contribution) at percentile p through the DESC-sorted list.
    p should be between 0 and 1.
    """
    N = df.shape[0]
    idx = int(p * (N - 1))
    return idx, df.iloc[idx]["Contribution"]

topK = [0.05 , 0.10, 0.12, 0.15, 0.18, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]

idx_list = []
percentage_list = []
contrib_list = []

for p in topK:
    idx, contrib = get_X_contribution(p)
    percentage_list.append(p)
    idx_list.append(idx)
    contrib_list.append(contrib)

# now build a DataFrame
pct_df = pd.DataFrame({
    "Percentage": percentage_list,
    "Data point":    idx_list,
    "Contribution": contrib_list
})

print(pct_df)



    Percentage  Data point  Contribution
0         0.05      117742      128417.0
1         0.10      235485       27144.0
2         0.12      282582       17610.0
3         0.15      353227       10179.0
4         0.18      423873        6438.0
5         0.20      470970        4902.0
6         0.30      706455        1577.0
7         0.40      941940         630.0
8         0.50     1177425         278.0
9         0.60     1412910         128.0
10        0.70     1648395          58.0
11        0.80     1883880          24.0
12        0.90     2119365           8.0


In [29]:
pct_df.to_csv("percentile_contribution.csv", index=False)

# Get 15% 

In [48]:
import pandas as pd

# compute how many rows = 15%
n = int(0.0002 * len(df))

# take the first n rows
top15pct = df.iloc[:n]

print(f"Taking {n} rows out of {len(df)} total → {len(df)} rows")
print(top15pct.head())
print(top15pct.shape)
top15pct.to_csv("./scratch/testTopContrib.csv", index=False)

Taking 70 rows out of 353227 total → 353227 rows
   Unnamed: 0 type  layer class1       class2  Neuron_ID  Contribution
0           0  mlp      7   math  programming       6117  8.144039e+11
1           1  mlp     24   math  programming       9329  7.782393e+11
2           2  mlp     23   math  programming      10184  7.565522e+11
3           3  res      1   math  programming      12054  6.470184e+11
4           4  mlp     24   math  programming        282  5.486137e+11
(70, 7)


In [5]:
# Get bottom rows 
# Option 1: using tail()
bottom50 = df.tail(50)

# Option 2: using iloc with negative indexing
bottom50 = df.iloc[-50:]

print(f"Taking last 50 rows out of {len(df)} total → {len(bottom50)} rows")
print(bottom50.head())
print(bottom50.shape)

# Save to CSV
bottom50.to_csv("./scratch/testBottomContrib.csv", index=False)

Taking last 50 rows out of 2354851 total → 50 rows
         Unnamed: 0 type  layer               class1       class2  Neuron_ID  \
2354801     2354801  mlp      5                 math  programming       5890   
2354802     2354802  mlp      5                 math  programming       5744   
2354803     2354803  mlp     24                 mmlu  programming       1690   
2354804     2354804  mlp     19                 math         mmlu       4195   
2354805     2354805  res      7  empathetic_dialogue         mmlu       7507   

         Contribution  
2354801           1.0  
2354802           1.0  
2354803           1.0  
2354804           1.0  
2354805           1.0  
(50, 7)


# Use Auto Interpret to get the neurons 

In [6]:
path = "./scratch/testBottomContrib.csv"
df = pd.read_csv(path, encoding="utf-8-sig")
df.head()

,Unnamed: 0,type,layer,class1,class2,Neuron_ID,Contribution
0,2354801,mlp,5,math,programming,5890,1.0
1,2354802,mlp,5,math,programming,5744,1.0
2,2354803,mlp,24,mmlu,programming,1690,1.0
3,2354804,mlp,19,math,mmlu,4195,1.0
4,2354805,res,7,empathetic_dialogue,mmlu,7507,1.0


In [7]:
df.shape

(50, 7)

In [8]:
import requests

def get_feature(model_id, source, index):
    try:
        url = f"https://www.neuronpedia.org/api/feature/{model_id}/{source}/{index}"
        resp = requests.get(url)
        resp.raise_for_status()            # throws if not 200
        feature = resp.json()
        
        # 2. Grab the list of explanations (might be empty!)
        explanations = feature.get("explanations", [])
        
        # 3. first description:
        # if explanations:
        return explanations[0]["description"]
    except Exception as e:
        return None
    
for i in range(10):
    model = "gemma-2-2b"
    layer = 21
    sae_type = "mlp"
    source = f"{layer}-gemmascope-{sae_type}-16k"
    feat = get_feature(model, source, i)
    print(feat)

 instances of parentheses and expressions of the concept of "status quo."
elements of HTML and programming language syntax
 references to programming methods and serialization processes
 programming loops and function definitions
references to SEO practices and related features in marketing contexts
 phrases expressing possession or ownership
aspirations related to careers in design and cuisine
references to specific individuals and their roles in a legal context
 references to specific objects or tools in a narrative context
 phrases indicating duration or quantity of time


In [9]:
from tqdm.auto import tqdm

model = "gemma-2-2b"
explanations = []

# for idx, row in df.iterrows():
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Fetching features"):
    sae_type  = row["type"]
    layer     = row["layer"]
    neuron_id = row["Neuron_ID"]
    source    = f"{layer}-gemmascope-{sae_type}-16k"

    feat = get_feature(model, source, neuron_id)
    explanations.append(feat)
    # print(f"{idx:>6}: type={sae_type}, layer={layer}, neuron={neuron_id} → {feat!r}")
df["Explanation"] = explanations
df.to_csv(path, index=False, encoding="utf-8-sig")

Fetching features:   0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
import pandas as pd

# load and drop the Unnamed:0 column
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

# convert to LaTeX and strip off the \begin{tabular}/\toprule/... wrapper
latex = df.to_latex(index=False, escape=False, column_format="lllllll")
lines = latex.splitlines()

# the data rows live between the header (line 3) and the footer (last 2 lines)
body = lines[3:-2]

# print each row, adding a \\ at the end
for row in body:
    print(row.rstrip() + "")


\midrule
mlp & 5 & math & programming & 5890 & 1.000000 &  structures related to providing reasons or tips \\
mlp & 5 & math & programming & 5744 & 1.000000 &  key-value assignments or variable declarations in programming contexts \\
mlp & 24 & mmlu & programming & 1690 & 1.000000 & references to educational qualifications and assessments \\
mlp & 19 & math & mmlu & 4195 & 1.000000 &  proper names and titles, particularly related to individuals and their credentials \\
res & 7 & empathetic_dialogue & mmlu & 7507 & 1.000000 & references to sports team management and player transfers \\
mlp & 5 & math & programming & 6518 & 1.000000 & details relating to interpersonal interactions and feelings \\
res & 23 & mmlu & programming & 5989 & 1.000000 &  the word "por" and its variations \\
mlp & 10 & math & programming & 15314 & 1.000000 &  HTML closing tags and associated text elements, particularly in a UI context \\
att & 8 & mmlu & programming & 10416 & 1.000000 &  attends to vegetarian-rel